In [1]:
import os
import pyspark
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
spark

In [2]:
#df = spark.read.\
#format("json").load("flight-data/json/2015-summary.json")
#df.printSchema()

In [3]:
#df = spark.read.format("csv").option("header", True).option("InferSchema", True).load("retail-data/online-retail-dataset.csv").coalesce(5)

In [4]:
#df.count()

In [5]:
#df.printSchema()

In [6]:
#from pyspark.sql.functions import avg
#df.select(avg("UnitPrice")).show()

In [7]:
from pyspark.sql import Row

myRow = Row("Hello", None, 1, False)

In [8]:
myRow

<Row('Hello', None, 1, False)>

In [9]:
myRow[0]

'Hello'

In [10]:
myRow[1]

In [11]:
myRow[2]

1

In [12]:
myRow[3]

False

In [13]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, DoubleType

myManualSchema = StructType([
    StructField("some", StringType(), True),
    StructField("col", StringType(), True),
    StructField("names", LongType(), False) ])



In [14]:
myManualSchema

StructType([StructField('some', StringType(), True), StructField('col', StringType(), True), StructField('names', LongType(), False)])

In [15]:
myRow = Row("Hello", None, 1)
myRow1 = Row("Darkness", None, 1)
myRow2 = Row("My", None, 1)
myRow3 = Row("Old", None, 1)
myRow4 = Row("Friend", None, 1)
myDf = spark.createDataFrame([myRow,myRow1, myRow2, myRow3, myRow4], myManualSchema)


In [16]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [17]:
myDf.show()

+--------+----+-----+
|    some| col|names|
+--------+----+-----+
|   Hello|null|    1|
|Darkness|null|    1|
|      My|null|    1|
|     Old|null|    1|
|  Friend|null|    1|
+--------+----+-----+



In [18]:
from pyspark.sql.functions import *

In [19]:
df = spark.read.option("header", True).option("InferSchema", True).csv("flight-data/csv/2015-summary.csv").coalesce(5)

In [20]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [21]:
df.select(col("ORIGIN_COUNTRY_NAME").alias("Coutry")).distinct().toPandas()

,Coutry
0,Paraguay
1,Russia
2,Anguilla
3,Senegal
4,Sweden
...,...
120,Hungary
121,Pakistan
122,United Kingdom
123,Vietnam


In [22]:
df.limit(20).toPandas()

,DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
0,United States,Romania,15
1,United States,Croatia,1
2,United States,Ireland,344
3,Egypt,United States,15
4,United States,India,62
5,United States,Singapore,1
6,United States,Grenada,62
7,Costa Rica,United States,588
8,Senegal,United States,40
9,Moldova,United States,1


In [23]:
df.rdd.getNumPartitions()

1

In [24]:
df.repartition(5)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

In [25]:
df.rdd.getNumPartitions()

1